In [ ]:
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os

sys.path.append(os.path.abspath(".."))
fig_dir = '../outputs/figures'
os.makedirs(fig_dir, exist_ok=True)

BASE_DIR = Path().resolve().parent

DATA_DIR = BASE_DIR / "data"
RAW_DIR = DATA_DIR / "raw"
INTERIM_DIR = DATA_DIR / "interim"
PROCESSED_DIR = DATA_DIR / "processed"

#LOG_INT_PATH = INTERIM_DIR / "log_int.pkl"
#MART_INT_PATH = INTERIM_DIR / "mart_int.pkl"
TPS_INT_PATH = INTERIM_DIR / "tps_sample.csv"
FINAL_DATASET1_PATH = PROCESSED_DIR / "tps_mart_log1.csv"
FINAL_DATASET2_PATH = PROCESSED_DIR / "final1.pickle"
FINAL_DATASET3_PATH = PROCESSED_DIR / "total_sha_tps_cancel_2023.pkl"

MART_RAW_PATH = RAW_DIR / "mart_raw.csv"
LOG_RAW_PATH = RAW_DIR / "log_raw.csv"

MART_PATH = RAW_DIR / "mart.pkl"
LOG_PATH = RAW_DIR / "log.pkl"
TPS_PATH = RAW_DIR / "tps.pkl"

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False  



In [ ]:
tps = pd.read_pickle(FINAL_DATASET3_PATH)
tps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22893438 entries, 0 to 22893437
Data columns (total 39 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   sha2_hash                  object        
 1   SVC_USE_DAYS_GRP           category      
 2   MEDIA_NM_GRP               category      
 3   PROD_NM_GRP                category      
 4   PROD_OLD_YN                bool          
 5   PROD_ONE_PLUS_YN           bool          
 6   AGMT_KIND_NM               category      
 7   STB_RES_1M_YN              bool          
 8   SVOD_SCRB_CNT_GRP          category      
 9   PAID_CHNL_CNT_GRP          category      
 10  SCRB_PATH_NM_GRP           category      
 11  INHOME_RATE                object        
 12  AGMT_END_SEG               category      
 13  AGMT_END_YMD               datetime64[ns]
 14  TOTAL_USED_DAYS            int16         
 15  TV_SCRB                    int16         
 16  ANALOG_SCRB                int8   

In [6]:
tps_raw_path = "../data/raw/tps_raw.csv"
tps.to_csv(tps_raw_path)
print(f" 파일 저장 완료: {tps_raw_path}")

 파일 저장 완료: ../data/raw/tps_raw.csv


In [9]:
mart = pd.read_csv(MART_RAW_PATH,                       
                       sep = ",", 
                       encoding = 'utf-8', 
                       engine="python",
                       on_bad_lines="skip")
mart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470064 entries, 0 to 470063
Data columns (total 88 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   asset_id            470064 non-null  object 
 1   actr_disp           470060 non-null  object 
 2   asset_nm            470064 non-null  object 
 3   asset_prod          470064 non-null  object 
 4   aud                 0 non-null       float64
 5   audience_cnt        0 non-null       float64
 6   broad_ymd           466682 non-null  object 
 7   category            468827 non-null  object 
 8   chapter             468852 non-null  object 
 9   created             470064 non-null  int64  
 10  created_by          470064 non-null  object 
 11  crt_ymd             467690 non-null  object 
 12  ct_cl               470064 non-null  object 
 13  cts_id              468751 non-null  object 
 14  description         470064 non-null  object 
 15  director            470036 non-nul

In [10]:
mart_path = "../data/raw/mart.pkl"
mart.to_pickle(mart_path)
print(f" 파일 저장 완료: {mart_path}")

 파일 저장 완료: ../data/raw/mart.pkl


In [ ]:
log = pd.read_csv(LOG_RAW_PATH,                       
                       sep = ",", 
                       encoding = 'utf-8', 
                       engine="python",
                       on_bad_lines="skip")
log.info()

In [ ]:
logt_path = "../data/raw/log.pkl"
log.to_pickle(logt_path)
print(f" 파일 저장 완료: {logt_path}")

In [ ]:
tps = pd.read_pickle(TPS_PATH)
tps.info()
mart = pd.read_pickle(MART_PATH)
mart.info()
log = pd.read_pickle(LOG_PATH)
log.info()

In [ ]:

# ===== 2️⃣ STEP1: 분기 컬럼 생성 =====
# 월 추출
log['month'] = pd.to_datetime(log['strt_dt']).dt.month
tps['month'] = tps['p_mt']  # p_mt가 월 정보라면 그대로 사용

# 월 → 분기 매핑
def month_to_quarter(m):
    if m in [1,2,3]: return 'Q1'
    elif m in [4,5,6]: return 'Q2'
    elif m in [7,8,9]: return 'Q3'
    else: return 'Q4'

log['quarter'] = log['month'].apply(month_to_quarter)
tps['quarter'] = tps['month'].apply(month_to_quarter)

# ===== 3️⃣ STEP2: 고객별 분기 feature 생성 =====
# RVOD 관련 feature (로그 기준)
rvod_log = log[log['category']=='RVOD']  # RVOD만 추출
rvod_features = rvod_log.groupby(['sha2_hash','quarter']).agg(
    RVOD_cnt = ('asset_id','count'),
    RVOD_use_tms = ('use_tms','sum'),
    RVOD_disp_rtm = ('disp_rtm','sum')
).reset_index()

# SVOD 관련 feature (TPS 기준)
svod_features = tps.groupby(['sha2_hash','quarter']).agg(
    SVOD_cnt = ('SVOD_SCRB_CNT_GRP','sum'),
    SVOD_inhome_rate = ('INHOME_RATE','mean'),
    SVOD_used_days = ('TOTAL_USED_DAYS','sum')
).reset_index()

# TV/인터넷/번들 feature
tps_features = tps.groupby(['sha2_hash','quarter']).agg(
    TV_SCRB=('TV_SCRB','sum'),
    DIGITAL_SCRB=('DIGITAL_SCRB','sum'),
    BUNDLE_YN=('BUNDLE_YN','max'),   # boolean 최빈값 또는 max
    CH_HH_avg=('CH_HH_AVG_MONTH1','mean')
).reset_index()

# 장르 선호 feature (로그+MART 활용)
log_mart = log.merge(mart[['full_asset_id','genre_of_ct_cl']], left_on='asset(', right_on='full_asset_id', how='left')
genre_features = log_mart.groupby(['sha2_hash','quarter','genre_of_ct_cl']).agg(
    genre_watch_cnt=('asset_id','count'),
    genre_watch_tms=('use_tms','sum')
).reset_index()
# 장르 pivot
genre_features = genre_features.pivot_table(
    index=['sha2_hash','quarter'], 
    columns='genre_of_ct_cl', 
    values=['genre_watch_cnt','genre_watch_tms'], 
    fill_value=0
)
# 컬럼 이름 단순화
genre_features.columns = ['_'.join(col).strip() for col in genre_features.columns.values]
genre_features = genre_features.reset_index()

# ===== 4️⃣ STEP2-2: 고객 feature 통합 =====
features = rvod_features.merge(svod_features, on=['sha2_hash','quarter'], how='outer') \
                        .merge(tps_features, on=['sha2_hash','quarter'], how='outer') \
                        .merge(genre_features, on=['sha2_hash','quarter'], how='outer')

# ===== 5️⃣ STEP1-2: 결측치 보간 (분기 평균으로) =====
num_cols = features.select_dtypes(include=np.number).columns.tolist()
num_cols.remove('quarter') if 'quarter' in num_cols else None

# 분기별 평균으로 결측치 채우기
features[num_cols] = features.groupby('quarter')[num_cols].transform(lambda x: x.fillna(x.mean()))

# 범주형 결측치는 최빈값으로 채우기
cat_cols = features.select_dtypes(include='object').columns.tolist()
cat_cols.remove('sha2_hash')
cat_cols.remove('quarter')
for c in cat_cols:
    features[c] = features[c].fillna(features[c].mode()[0])

# ===== 6️⃣ STEP3: 다음 분기 RVOD target 생성 =====
features = features.sort_values(['sha2_hash','quarter'])
features['RVOD_next_Qtr'] = features.groupby('sha2_hash')['RVOD_cnt'].shift(-1)

# RVOD_next_Qtr가 target, 나머지가 feature
# 필요 시 결측치는 0으로 처리 (마지막 분기)
features['RVOD_next_Qtr'] = features['RVOD_next_Qtr'].fillna(0)

# ===== 7️⃣ 결과 확인 =====
print(features.head())
print("컬럼:", features.columns.tolist())


In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score

# ===== 데이터 로드 (STEP1~2 완료된 feature 테이블) =====
# features 테이블에는 sha2_hash, quarter, feature 컬럼들, RVOD_next_Qtr(target) 있음
# RVOD_next_Qtr는 해당 분기의 다음 분기 RVOD 구매 건수
features = pd.read_csv("features_table.csv")  # STEP1~2 완료 후 저장된 테이블

# 숫자형/범주형 구분
num_cols = features.select_dtypes(include=np.number).columns.tolist()
num_cols.remove('RVOD_next_Qtr')
num_cols.remove('quarter') if 'quarter' in num_cols else None
cat_cols = features.select_dtypes(include='object').columns.tolist()
cat_cols.remove('sha2_hash')
cat_cols.remove('quarter')

# ===== 범주형은 라벨 인코딩 =====
from sklearn.preprocessing import LabelEncoder
for c in cat_cols:
    le = LabelEncoder()
    features[c] = le.fit_transform(features[c].astype(str))

# ===== 1️⃣ 분기 순서 지정 =====
quarter_order = ['Q1','Q2','Q3','Q4']

# 결과 저장용
rolling_results = []

# ===== 2️⃣ Rolling 학습 & 예측 =====
for i in range(len(quarter_order)-1):  # Q1→Q2, Q2→Q3, Q3→Q4
    train_q = quarter_order[i]
    test_q = quarter_order[i+1]

    train_data = features[features['quarter']==train_q]
    test_data = features[features['quarter']==test_q]

    X_train = train_data[num_cols + cat_cols]
    y_train = train_data['RVOD_next_Qtr']

    X_test = test_data[num_cols + cat_cols]
    y_test = test_data['RVOD_next_Qtr']

    # LightGBM Dataset
    lgb_train = lgb.Dataset(X_train, label=y_train)
    lgb_test = lgb.Dataset(X_test, label=y_test, reference=lgb_train)

    # 모델 학습
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'learning_rate': 0.05,
        'num_leaves': 31,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'seed': 42
    }
    model = lgb.train(params, lgb_train, num_boost_round=500,
                      valid_sets=[lgb_train, lgb_test],
                      early_stopping_rounds=50, verbose_eval=50)

    # 예측
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    # 평가
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)

    print(f"{train_q} → {test_q} : RMSE={rmse:.4f}, R2={r2:.4f}")

    # 결과 저장
    temp = test_data[['sha2_hash','quarter']].copy()
    temp['RVOD_pred'] = y_pred
    rolling_results.append(temp)

# ===== 3️⃣ 내년 Q1 예측 =====
# 내년 Q1은 올해 Q4 데이터를 feature로 사용
latest_q = 'Q4'
latest_data = features[features['quarter']==latest_q]

X_nextQ = latest_data[num_cols + cat_cols]

# Q4 모델 그대로 사용하거나, Q1~Q4 합쳐서 전체 학습 후 내년 Q1 예측 가능
# 여기서는 가장 최근 Q4 모델 사용
y_nextQ_pred = model.predict(X_nextQ, num_iteration=model.best_iteration)

nextQ_pred = latest_data[['sha2_hash']].copy()
nextQ_pred['RVOD_next_year_Q1_pred'] = y_nextQ_pred

print(nextQ_pred.head())

# ===== 4️⃣ Rolling 결과 합치기 =====
rolling_df = pd.concat(rolling_results, ignore_index=True)
print(rolling_df.head())


In [ ]:
import matplotlib.pyplot as plt
import lightgbm as lgb

# 모델 학습 후
lgb.plot_importance(model, max_num_features=20, importance_type='gain', figsize=(10,6))
plt.show()
